In [19]:
import pandas as pd
import datetime
import numpy as np

# csv 파일 불러오기

In [2]:
booking = pd.read_csv('./data/booking_data.csv')
cooking = pd.read_csv('./data/Cooking_data_set.csv')
error = pd.read_csv('./data/Error_Message.csv')
product = pd.read_csv('./data/Product_Name.csv')

C:\Users\mazy0\AppData\Local\Temp\ipykernel_49824\3866819654.py:1: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  booking = pd.read_csv('./data/booking_data.csv')


# 데이터 전처리

## NaN 제거
- 품목코드 3개 없는 것 삭제

In [3]:
# NaN 제거
error_set = error.dropna()

## 발생시간, 종료시간, 오류조치시간

### 발생시간, 종료시간 datetime 형변환

In [32]:
error_set['발생시간'] = pd.to_datetime(error_set['발생시간'], format='%H:%M')
error_set['종료시간'] = pd.to_datetime(error_set['종료시간'], format='%H:%M')

C:\Users\mazy0\AppData\Local\Temp\ipykernel_49824\3354423844.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  error_set['발생시간'] = pd.to_datetime(error_set['발생시간'], format='%H:%M')
C:\Users\mazy0\AppData\Local\Temp\ipykernel_49824\3354423844.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  error_set['종료시간'] = pd.to_datetime(error_set['종료시간'], format='%H:%M')


In [12]:
error_set.dtypes

순번                        int64
생산라인코드                   object
생산라인명                    object
생산일자                     object
발생시간             datetime64[ns]
종료시간             datetime64[ns]
오류조치시간                    int64
비가동코드                    object
Error_Message            object
품목코드                     object
품목명                      object
오류발생작업장명                 object
dtype: object

### 오류조치시간 재계산

In [33]:
error_set_minus = error_set[pd.to_datetime(error_set['종료시간'], format='%H:%M') < pd.to_datetime(error_set['발생시간'], format='%H:%M')]
error_set_minus['종료시간'] = pd.to_datetime(error_set_minus['종료시간'], format='%H:%M') + datetime.timedelta(days=1)

In [23]:
idx = error_set_minus.index

error_set.loc[idx,'종료시간'] = error_set_minus.loc[idx,'종료시간']

In [24]:
error_set['오류조치시간'] = (pd.to_datetime(error_set['종료시간']).astype(np.int64) - pd.to_datetime(error_set['발생시간']).astype(np.int64)) // 60000000000

C:\Users\mazy0\AppData\Local\Temp\ipykernel_49824\159894302.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  error_set['오류조치시간'] = (pd.to_datetime(error_set['종료시간']).astype(np.int64) - pd.to_datetime(error_set['발생시간']).astype(np.int64)) // 60000000000


In [25]:
error_set

순번 생산라인코드      생산라인명        생산일자                발생시간  \
0     10   D004       PET병  2011-01-03 1900-01-01 14:16:00   
1      7   D011    2동 오리히로  2011-01-04 1900-01-01 18:54:00   
2      2   C002      쿠커 #2  2011-01-05 1900-01-01 09:51:00   
3     13   P004      PET#1  2011-01-05 1900-01-01 18:20:00   
4     17   C004      쿠커 #4  2011-01-10 1900-01-01 22:32:00   
...   ..    ...        ...         ...                 ...   
1068   2  D0036     로타리 #5  2021-10-22 1900-01-01 12:30:00   
1069   3   D084   2동 로타리#4  2021-10-25 1900-01-01 09:46:00   
1070   1   C045      쿠커 #5  2021-10-25 1900-01-01 08:58:00   
1072   1   D005      3.2마요  2021-10-28 1900-01-01 10:50:00   
1073   7   D087  냉동밥 내포장#2  2021-10-28 1900-01-01 14:15:00   

                    종료시간  오류조치시간    비가동코드 Error_Message      품목코드  \
0    1900-01-01 14:24:00       8  W0056B4          위치조정  13D02080   
1    1900-01-01 19:03:00       9  W005711            대기  16A01016   
2    1900-01-01 09:58:00       7  W003452            충전  23K01321   
3    1900-01-01 20:44:00     144  W0076B2           구동부  13R02051   
4    1900-01-02 01:16:00     164  W003721        충진 미종료  23D02080   
...                  ...     ...      ...           ...       ...   
1068 1900-01-01 18:34:00     364  W005712            고장  16B06061   
1069 1900-01-01 11:49:00     123  W005713            수리  13A00725   
1070 1900-01-01 09:04:00       6  W003446         ph 이상  23U02003   
1072 1900-01-01 11:06:00      16  W0058Z9            기타  16B06099   
1073 1900-01-01 14:35:00      20  W005722            공정  1BA03638   

                                          품목명 오류발생작업장명  
0                                    참깨흑임자드레싱      충전실  
1                                 시아스고구마샐러드믹스      충전실  
2                                  나가사키베이스(재)      쿠킹실  
3                                 이마트베스트돈까스소스      포장실  
4                                 참깨흑임자드레싱(재)      쿠킹실  
...                                       ...      ...  
1068                            천년풍미퓨어마요네즈1kg      충전실  
1069                                  샘표강된장양념      충전실  
1070                       유기아가베시럽665g/올가홀(재)      쿠킹실  
1072                           맛있는골드마요네즈3.2kg      충전실  
1073  (1)Homeplus signature 날치알이톡톡터지는날치알밥(알밥)      충전실  

[1071 rows x 12 columns]

## cooking과 error 합치기

In [29]:
cooking.sample()

순번 생산라인코드  생산라인명        생산일자      품목코드                         품목명  \
20988   2   C004  쿠커 #4  2020-02-24  23D01324  CITRON DRESSING(냉장출고보관)(재)   

      작업장코드    충전실온도     실링온도      쿠킹온도  쿠킹스팀압력     실링압력   생산시간  
20988  W003  71.7303  92.2303  127.8738  23.762  207.721  71.94

In [37]:
# 생산시간이 없는 row만 가져온다.
cooking_notime = cooking[cooking['생산시간'].isnull()]
# cooking_notime['생산시간'] = cooking_notime['생산시간'].fillna(0)

# 생산시간 column 삭제 시도
cooking_notime = cooking_notime.drop(['생산시간'], axis=1)

In [38]:
cooking_notime

순번 생산라인코드      생산라인명        생산일자      품목코드  \
10     10   D004       PET병  2011-01-03  13D02080   
43      7   D011    2동 오리히로  2011-01-04  16A01016   
47      2   C002      쿠커 #2  2011-01-05  23K01321   
71     13   P004      PET#1  2011-01-05  13R02051   
125    17   C004      쿠커 #4  2011-01-10  23D02080   
...    ..    ...        ...         ...       ...   
24030   3   D084   2동 로타리#4  2021-10-25  13A00725   
24032   1   C045      쿠커 #5  2021-10-25  23U02003   
24049   7   D088  냉동밥 내포장#1  2021-10-28  1BA03709   
24056   1   D005      3.2마요  2021-10-28  16B06099   
24060   7   D087  냉동밥 내포장#2  2021-10-28  1BA03638   

                                           품목명 작업장코드      충전실온도        실링온도  \
10                                    참깨흑임자드레싱  W005  71.125831   93.425831   
43                                 시아스고구마샐러드믹스  W005  71.587470  103.987470   
47                                  나가사키베이스(재)  W003  70.540995  102.440995   
71                                 이마트베스트돈까스소스  W007  71.401189  111.901189   
125                                참깨흑임자드레싱(재)  W003  70.649333  105.149333   
...                                        ...   ...        ...         ...   
24030                                  샘표강된장양념  W005  71.671267  112.571266   
24032                       유기아가베시럽665g/올가홀(재)  W003  69.827929   86.927929   
24049                                      NaN  W005  72.585609   93.885609   
24056                           맛있는골드마요네즈3.2kg  W005  71.764168  108.664168   
24060  (1)Homeplus signature 날치알이톡톡터지는날치알밥(알밥)  W005  70.610616  103.710616   

             쿠킹온도  쿠킹스팀압력     실링압력  
10     127.878281  22.968  201.430  
43     138.736120  23.166  204.974  
47     137.135195  23.129  204.249  
71     146.585739  23.123  200.068  
125    139.680183  23.021  200.541  
...           ...     ...      ...  
24030  147.339967  23.179  211.102  
24032  121.388229  22.974  200.317  
24049  128.549109  23.109  204.160  
24056  142.862968  22.799  201.039  
24060  138.257466  23.031  199.343  

[1074 rows x 12 columns]

In [28]:
cooking_notime = cooking_notime.dropna()

In [30]:
error_cooking = error_set.merge(cooking_notime, left_on=['순번','생산라인코드','생산라인명','생산일자','품목코드','품목명'],
right_on=['순번','생산라인코드','생산라인명','생산일자','품목코드','품목명'])

In [31]:
error_cooking

순번 생산라인코드      생산라인명        생산일자                발생시간  \
0     10   D004       PET병  2011-01-03 1900-01-01 14:16:00   
1      7   D011    2동 오리히로  2011-01-04 1900-01-01 18:54:00   
2      2   C002      쿠커 #2  2011-01-05 1900-01-01 09:51:00   
3     13   P004      PET#1  2011-01-05 1900-01-01 18:20:00   
4     17   C004      쿠커 #4  2011-01-10 1900-01-01 22:32:00   
...   ..    ...        ...         ...                 ...   
1068   2  D0036     로타리 #5  2021-10-22 1900-01-01 12:30:00   
1069   3   D084   2동 로타리#4  2021-10-25 1900-01-01 09:46:00   
1070   1   C045      쿠커 #5  2021-10-25 1900-01-01 08:58:00   
1071   1   D005      3.2마요  2021-10-28 1900-01-01 10:50:00   
1072   7   D087  냉동밥 내포장#2  2021-10-28 1900-01-01 14:15:00   

                    종료시간  오류조치시간    비가동코드 Error_Message      품목코드  \
0    1900-01-01 14:24:00       8  W0056B4          위치조정  13D02080   
1    1900-01-01 19:03:00       9  W005711            대기  16A01016   
2    1900-01-01 09:58:00       7  W003452            충전  23K01321   
3    1900-01-01 20:44:00     144  W0076B2           구동부  13R02051   
4    1900-01-02 01:16:00     164  W003721        충진 미종료  23D02080   
...                  ...     ...      ...           ...       ...   
1068 1900-01-01 18:34:00     364  W005712            고장  16B06061   
1069 1900-01-01 11:49:00     123  W005713            수리  13A00725   
1070 1900-01-01 09:04:00       6  W003446         ph 이상  23U02003   
1071 1900-01-01 11:06:00      16  W0058Z9            기타  16B06099   
1072 1900-01-01 14:35:00      20  W005722            공정  1BA03638   

                                          품목명 오류발생작업장명 작업장코드      충전실온도  \
0                                    참깨흑임자드레싱      충전실  W005  71.125831   
1                                 시아스고구마샐러드믹스      충전실  W005  71.587470   
2                                  나가사키베이스(재)      쿠킹실  W003  70.540995   
3                                 이마트베스트돈까스소스      포장실  W007  71.401189   
4                                 참깨흑임자드레싱(재)      쿠킹실  W003  70.649333   
...                                       ...      ...   ...        ...   
1068                            천년풍미퓨어마요네즈1kg      충전실  W005  71.865935   
1069                                  샘표강된장양념      충전실  W005  71.671267   
1070                       유기아가베시럽665g/올가홀(재)      쿠킹실  W003  69.827929   
1071                           맛있는골드마요네즈3.2kg      충전실  W005  71.764168   
1072  (1)Homeplus signature 날치알이톡톡터지는날치알밥(알밥)      충전실  W005  70.610616   

            실링온도        쿠킹온도  쿠킹스팀압력     실링압력  생산시간  
0      93.425831  127.878281  22.968  201.430   0.0  
1     103.987470  138.736120  23.166  204.974   0.0  
2     102.440995  137.135195  23.129  204.249   0.0  
3     111.901189  146.585739  23.123  200.068   0.0  
4     105.149333  139.680183  23.021  200.541   0.0  
...          ...         ...     ...      ...   ...  
1068  105.165935  139.863435  23.132  202.605   0.0  
1069  112.571266  147.339967  23.179  211.102   0.0  
1070   86.927929  121.388229  22.974  200.317   0.0  
1071  108.664168  142.862968  22.799  201.039   0.0  
1072  103.710616  138.257466  23.031  199.343   0.0  

[1073 rows x 19 columns]